In [ ]:
from transformers import BertModel, AutoTokenizer, pipeline
from datasets import load_dataset
import torch

In [ ]:
model_name = "bert-base-cased"
device = "cpu"

model = BertModel.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model_inputs = tokenizer("My name is Bert", return_tensors="pt")

In [ ]:
model_inputs_on_device = {arg_name: tensor.to(device) for arg_name, tensor in model_inputs.items()}

In [ ]:
for _ in range(10):
    model(**model_inputs_on_device);

In [ ]:
%%timeit -r 30
model(**model_inputs_on_device);

352 ms ± 34.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [ ]:
model_qt = torch.quantization.quantize_dynamic(
                    model=model,
                    qconfig_spec = {
                        torch.nn.Linear : torch.quantization.default_dynamic_qconfig,
                    },
                    dtype=torch.qint8,
                )

In [ ]:
for _ in range(10):
    model_qt(**model_inputs_on_device);

In [ ]:
%%timeit -r 30
model_qt(**model_inputs_on_device);

345 ms ± 33.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [ ]:
model.to('cpu');

In [ ]:
ds = load_dataset('squad')

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)


In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
sample = ds['train'][0]
sample

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [ ]:
pipe = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [ ]:
%%timeit
pipe(question=sample['question'], context=sample['context'])

315 ms ± 72.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
model_qt = torch.quantization.quantize_dynamic(
                    model=model,
                    qconfig_spec = {
                        torch.nn.Linear : torch.quantization.default_dynamic_qconfig,
                    },
                    dtype=torch.qint8,
                )

In [ ]:
pipe_qt = pipeline('question-answering', model=model_qt, tokenizer=tokenizer)

In [ ]:
%%timeit
pipe_qt(question=sample['question'], context=sample['context'])

380 ms ± 85.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
